# Generate data from paper figures

In [ ]:
import holoviews as hv
import matplotlib.pyplot as plt
import os
hv.notebook_extension()
%matplotlib inline
%output fig='svg'
%opts Path [aspect='square'] (lw=0.5) Image [aspect='square' colorbar=True] (cmap='inferno_r')
os.makedirs('new_data', exist_ok=True) # Create new dir for generated data

In [ ]:
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    from hpc05 import Client as Client
except ImportError:
    print("Start your ipcluster in the IPython Clusters tab if you haven't yet!")
    from ipyparallel import Client
    
client = Client()

In [ ]:
dview = client[:]
lview = client.load_balanced_view()
print('Connected to {} engines.'.format(len(dview)))

In [ ]:
%%px --local
import sys
# The cluster needs to know where shortjunction.py is located.
sys.path.append('/home/basnijholt/Dropbox/Work/short_jj_majorana/Code/')
sys.path.append('/home/jovyan/work/') # Location for Docker, see README.md
import kwant
import numpy as np
import sympy
from shortjunction import (NS_infinite_2D_3D, slowest_evan_mode, find_gap, 
                           energies_over_delta, make_params, plot_phase,
                           plot_decay_lengths,SimpleNamespace, Ez_to_B,
                           constants, sparse_eigs, NS_finite, NS_infinite,
                           SNS_infinite, s0, sx, sy, sz)

# NS infinite
Fig. 2

In [ ]:
def NS_infinite_spectrum(Ezs, wire_params, p, fname):
    """Computes the Andreev spectrum of a finite NS-junction
    as a function of magnetic field.
    
    Parameters:
    -----------
    Ezs : list or numpy.ndarray
        Sequence of Zeeman energies at which the spectrum is calculated.
    wire_params : dict
        Dictionary containing the parameters used to create the Kwant system.
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    fname : str
        Filename of the data to be stored.
        
    Returns:
    --------
    energies : numpy.array
        Array containing the energies over the superconducting gap.
    """
    dview['wire_params'] = wire_params
    %px syst = NS_infinite(**wire_params)
    energies = lview.map_async(lambda Ez: energies_over_delta(syst, p.update(Ez=Ez), k_x=0), Ezs)
    energies.wait_interactive()
    energies = np.array(energies.result()).reshape(-1, 2)    
    np.savetxt(fname, energies)
    return energies

p = SimpleNamespace(Ez=None, delta=0.25, alpha=20, t=constants.t,
                    tpar=0.1*constants.t, mu=3, kx=0)

wire_params = dict(a=0.5, L=200)
Ezs = np.linspace(1e-3, 2.5, 200)
energies = NS_infinite_spectrum(Ezs, wire_params, p, fname='new_data/NS_infinite.npz')

In [ ]:
%matplotlib inline
plt.figure(figsize=(8, 7))
plt.xlabel(r'$B$ (meV)')
plt.ylabel(r'$E / \Delta$')
plt.plot(Ezs, energies, 'b-', lw=2.)
plt.grid()
plt.show()

# 2D decay lengths
Fig. 6

Note that this calculates the phase diagram on a lower resolution than in the paper.

In [ ]:
%%opts Image (clims=(0, 4000))
p = SimpleNamespace(mu_sc=8, orbital=False, Delta=0.25, alpha=20, B_y=0, B_z=0,
                    mu_B=constants.mu_B, t=constants.t, g=constants.g)

Bs = np.linspace(0, 5, 200)
mus = np.linspace(1, 10, 200)

decay_length_2d = plot_decay_lengths(dview, lview, p, W=100, H=None,
                                     Bs=Bs, mus=mus, dim=2, async_parallel=False)
hv.Store.dump(decay_length_2d,
              open('new_data/decay_length_2d.h', 'wb'))
decay_length_2d

# NS finite
Fig. 8

Takes ~10 minutes on 140 cores with current parameters.


In [ ]:
def NS_finite_spectrum(Ezs, wire_params, p, fname):
    """Computes the spectrum of a finite NS-junction
    as a function of magnetic field.
    
    Parameters:
    -----------
    Ezs : list or numpy.ndarray
        Sequence of Zeeman energies at which the spectrum is calculated.
    wire_params : dict
        Dictionary containing the parameters used to create the Kwant system.
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    fname : str
        Filename of the data to be stored.
        
    Returns:
    --------
    energies : numpy.array
        Array containing the energies over the superconducting gap.
    """
    dview['wire_params'] = wire_params
    %px syst = NS_finite(**wire_params)
    energies = lview.map_async(lambda Ez: sparse_eigs(syst.hamiltonian_submatrix(
        sparse=True, args=[p.update(Ez=Ez)]), n_eigs=20, n_vec_lanczos=100), Ezs)
    energies.wait_interactive()
    energies = np.array(energies.result())
    np.savetxt(fname, energies / p.delta)
    return energies / p.delta


p = SimpleNamespace(Ez=None, delta=0.25, alpha=20, mu=3,
                    t=constants.t, tpar=0.1*constants.t)

wire_params = dict(a=10, L=300, W=10, W_sc=140)
Ezs = np.linspace(0, 8, 250)
energies = NS_finite_spectrum(Ezs, wire_params, p, fname='new_data/NS_finite.npz')

In [ ]:
plt.figure(figsize=(8, 7))
plt.plot(Ez_to_B(Ezs), energies, 'ro', ms=2.5, mec='None')
plt.xlabel(r'$B$ (meV)')
plt.ylabel(r'$E / \Delta$')
plt.ylim(0, 1)
plt.show()

# SNS infinite
Fig. 10

In [ ]:
def SNS_infinite_spectrum(Bs, wire_params, p, fname):
    """Computes the spectrum of a 2D SNS-junction
    as a function of magnetic field.
    
    Parameters:
    -----------
    Bs : list or numpy.ndarray
        Sequence of Zeeman energies at which the spectrum is calculated.
    wire_params : dict
        Dictionary containing the parameters used to create the Kwant system.
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    fname : str
        Filename of the data to be stored.
        
    Returns:
    --------
    energies : numpy.array
        Array containing the energies over the superconducting gap.
    """
    dview['wire_params'] = wire_params
    %px syst = SNS_infinite(**wire_params)
    energies = lview.map_async(lambda B: sparse_eigs(syst.hamiltonian_submatrix(
                sparse=True, args=[p.update(B=B), 0]), n_eigs=4, n_vec_lanczos=200), Bs)
    energies.wait_interactive()
    energies = np.array(energies.result())
    energies = np.array([np.min(np.abs(energies), axis=1),
                         np.max(np.abs(energies), axis=1)]) / p.delta
    np.savetxt(fname, energies)
    return energies


p = SimpleNamespace(B=0, delta=0.25, alpha=20, t=constants.t, tpar=constants.t, mu=3, D=200)
wire_params = dict(L=6000, W_sm=200, a=0.5)
Bs = np.linspace(0, 1.5, 140)
energies = SNS_infinite_spectrum(Bs, wire_params, p, 'new_data/SNS_infinite.npz')

In [ ]:
%matplotlib inline
plt.figure(figsize=(8, 7))
plt.plot(Bs, energies[0], '-ro', ms=4.5)
plt.plot(Bs, energies[1], '-ro', ms=4.5)
plt.xlabel(r'$B$ (T)')
plt.ylabel(r'$E / \Delta$')
plt.ylim(0, 1)
plt.xlim(0, max(Bs))
plt.grid()
plt.show()

# 3D phase diagram
Fig. 11

Note that this calculates the phase diagram on a lower resolution than in the paper.

In [ ]:
# Set the resolution of the plot in `kwargs`. Read the doc-strings of `plot_phase`.
kwargs = dict(dim=3, num_k=201, Bs=np.linspace(0, 1, 50), mus=np.linspace(0.1, 15, 50), async_parallel=True)
p = SimpleNamespace(mu_sc=8, Delta=0.25, alpha=20, B_y=0, B_z=0,
                    mu_B=constants.mu_B, t=constants.t, g=constants.g)

In [ ]:
p.orbital = False
phase_diagram_3d_no_orbital_100x100 = plot_phase(
    dview, lview, p, W=100, H=100,
    **kwargs).relabel('3D no orbital, $H=100, W=100, \mu_{sc}=\mu_{sm}$')
hv.Store.dump(phase_diagram_3d_no_orbital_100x100,
              open('new_data/phase_diagram_3d_no_orbital_100x100.h', 'wb'))
phase_diagram_3d_no_orbital_100x100

In [ ]:
p.orbital = False
phase_diagram_3d_no_orbital_120x120 = plot_phase(
    dview, lview, p, W=120, H=120,
    **kwargs).relabel('3D no orbital, $H=120, W=120, \mu_{sc}=\mu_{sm}$')
hv.Store.dump(phase_diagram_3d_no_orbital_120x120,
              open('new_data/phase_diagram_3d_no_orbital_120x120.h', 'wb'))
phase_diagram_3d_no_orbital_120x120

In [ ]:
p.orbital = True
phase_diagram_3d_with_orbital_100x100 = plot_phase(
    dview, lview, p, W=100, H=100,
    **kwargs).relabel('3D with orbital, $H=100, W=100, \mu_{sc}=\mu_{sm}$')
hv.Store.dump(phase_diagram_3d_with_orbital_100x100,
              open('new_data/phase_diagram_3d_with_orbital_100x100.h', 'wb'))
phase_diagram_3d_with_orbital_100x100

In [ ]:
p.orbital = True
phase_diagram_3d_with_orbital_120x120 = plot_phase(
    dview, lview, p, W=120, H=120, **kwargs).relabel('3D with orbital, $H=120, W=120, \mu_{sc}=\mu_{sm}$')

hv.Store.dump(phase_diagram_3d_with_orbital_120x120,
              open('new_data/phase_diagram_3d_with_orbital_120x120.h', 'wb'))
phase_diagram_3d_with_orbital_120x120